In [1]:
from collections import Counter
import sys, re


In [2]:
sys.argv = ["", "news-corpus-500k.txt", "questions.txt"]
context = []
word_context = []
with open(sys.argv[1], "r") as fh:
    for line in fh:
        words = ["<s>"]+[w for w in line.split(" ") if re.search('\w', w)]+["</s>"]
        context += words
        word_context += [" ".join([words[i+j] for j in range(2)]) for i in range(len(words)-1)]


In [13]:
print(list(context)[300:400])

['set', 'It', 'can', 'involve', 'thousands', 'tonnes', 'construction', 'waste', 'cost', 'millions', 'pounds', 'clear', 'up', 'Indonesia', 'example', 'are', '17,000', 'madrassas--and', 'local', 'NGO', 'Conservation', 'Religion', 'Initiative', 'reports', 'good', 'progress', 'persuading', 'teachers', 'those', 'schools', 'preach', 'practise', 'stewardship', 'NEW', 'YORK', 'AP', 'Pfizer', 'Inc', 'world', 'biggest', 'drug', 'maker', 'revenue', 'focus', 'recent', 'deals', 'new', 'research', 'data', 'integration', 'Wyeth', 'which', 'it', 'bought', 'year', 'ago', 'third-quarter', 'results', 'before', 'stock', 'market', 'opens', 'As', 'East', 'I', 'jumped', 'four', 'hearts', 'preempt', 'South', 'had', '0-1-6-6', 'distribution', 'wouldn', "'t", 'shut', 'Even', 'though', 'were', 'both', 'made', 'activists', 'stake', 'battle', 'between', 'protesters', 'police', 'do', 'seem', 'show', 'tactics', 'escalating', 'tensions', 'another', 'part', 'city', 'Obama', 'has', 'proposed', 'subsidies']


In [3]:

context = Counter(context)
word_context = Counter(word_context)

total_words = sum(list(context.values()))

249860 3040651


In [5]:
def unigram(value):
    return context[value]/total_words
	#return context.get(value, 0)/total_words


In [6]:
print(unigram("through") > unigram("threw"))

True


In [7]:
def bigram(value, smoothing=0):
    try:
        return (word_context.get(value, 0)+smoothing)/(context.get(value.split(" ")[0], 0)+smoothing*(len(context)))
    except ZeroDivisionError:
        return 0


In [8]:
bigram("check she")

0.001366120218579235

In [75]:

def predict(p1, p2, a1, a2):
    if p1 > p2:
        return a1, p1, False
    elif p1 < p2:
        return a2, p2, False
    else:
        return a1, p2, True


In [76]:
predict(unigram("through"), unigram("threw"), "through", "threw")

('through', 0.0006337913624159014, False)

In [82]:
	
ra = ["whether", "through", "piece", "court", "allowed", "check", "hear", "cereal", "chews", "sell"]
ai, ca_u, ca_b, ca_ba = 0, 0, 0, 0
with open(sys.argv[2], "r") as fh:
    for line in fh:
        u1, u2, b1, b2, ba1, ba2 = 1, 1, 1, 1, 1, 1
        answer1, answer2 = line.split(":")[1].split("/")[0].strip(), line.split(":")[1].split("/")[1].strip()
        line = line.split(":")[0]
        words = ["<s>"]+[w for w in line.split(" ") if re.search('\w', w)]+["</s>"]
        bi_words = [" ".join([words[i+j] for j in range(2)]) for i in range(len(words)-1)]

        for word in words:
            u1 *= unigram(word.replace("____", answer1))
            u2 *= unigram(word.replace("____", answer2))

        for bi_word in bi_words:
            b1 *= bigram(bi_word.replace("____", answer1))
            b2 *= bigram(bi_word.replace("____", answer2))

            ba1 *= bigram(bi_word.replace("____", answer1), 1)
            ba2 *= bigram(bi_word.replace("____", answer2), 1)

        print("Unigram model: ", end="")
        ans, prob, tie = predict(u1, u2, answer1, answer2)
        if tie == True:
            if prob == 0: print("Incorrect Answer")
            else: 
                ca_u += 0.5
                print(ans)
        else:
            if ans == ra[ai]: ca_u += 1
            print(ans)

        print("Bigram model: ", end="")
        ans, prob, tie = predict(b1, b2, answer1, answer2)
        if tie == True:
            if prob == 0: print("Incorrect Answer")
            else: 
                ca_b += 0.5
                print(ans)
        else:
            if ans == ra[ai]: ca_b += 1
            print(ans)


        print("Bigram with add-1 smoothing model : ", end = "")
        ans, prob, tie = predict(ba1, ba2, answer1, answer2)
        if tie == True:
            if prob == 0: print("Incorrect Answer")
            else: 
                ca_ba += 0.5
                print(ans)
        else:
            if ans == ra[ai]: ca_ba += 1
            print(ans)

        ai+=1

        print("\n ")

Unigram model: whether
Bigram model: whether
Bigram with add-1 smoothing model : whether

 
Unigram model: through
Bigram model: through
Bigram with add-1 smoothing model : through

 
Unigram model: peace
Bigram model: piece
Bigram with add-1 smoothing model : piece

 
Unigram model: court
Bigram model: court
Bigram with add-1 smoothing model : court

 
Unigram model: allowed
Bigram model: allowed
Bigram with add-1 smoothing model : allowed

 
Unigram model: check
Bigram model: check
Bigram with add-1 smoothing model : check

 
Unigram model: here
Bigram model: hear
Bigram with add-1 smoothing model : hear

 
Unigram model: serial
Bigram model: Incorrect Answer
Bigram with add-1 smoothing model : cereal

 
Unigram model: choose
Bigram model: Incorrect Answer
Bigram with add-1 smoothing model : choose

 
Unigram model: Incorrect Answer
Bigram model: Incorrect Answer
Bigram with add-1 smoothing model : sell

 


In [83]:
print("Unigram percentage accuracy: ", (ca_u/len(ra)) * 100)
print("Bigram percentage accuracy: ", (ca_b/len(ra)) * 100)
print("Bigram with add-1 smoothing percentage accuracy: ", (ca_ba/len(ra)) * 100)

Unigram percentage accuracy:  50.0
Bigram percentage accuracy:  70.0
Bigram with add-1 smoothing percentage accuracy:  90.0
